In [1]:
import argparse
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from matplotlib import cm
from functools import partial
import random


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [46]:
# export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/raw'
export_ds = '/media/berlukas/Data/data/nuscenes/raw/'

n_scans = 500
all_sem_clouds = []
for i in range(0, n_scans):
    filename = f"{export_ds}/09/scan{i}.npy"
    all_sem_clouds.append(np.load(filename))

print(f'Loaded {len(all_sem_clouds)} clouds from disk.')
print(f'Shape of one cloud is {all_sem_clouds[0].shape}')

Loaded 500 clouds from disk.
Shape of one cloud is (101725, 5)


In [47]:
n_clouds = len(all_sem_clouds)
idx_sem = 4
unique_classes = set()
for i in tqdm(range(0, n_clouds)):
    curr = set(np.unique(all_sem_clouds[i][:,idx_sem]))
    unique_classes.update(curr)    
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset.")
print(f'They are: {unique_classes}')

  0%|          | 0/500 [00:00<?, ?it/s]

we have 27 classes in the dataset.
They are: {0.0, 1.0, 259.0, 10.0, 11.0, 15.0, 18.0, 20.0, 30.0, 31.0, 32.0, 40.0, 44.0, 48.0, 49.0, 50.0, 51.0, 52.0, 70.0, 71.0, 72.0, 80.0, 81.0, 99.0, 252.0, 253.0, 254.0}


In [48]:
def map_kitti_to_nuscenes_label(label):
    noise = [0, 1, 11, 15, 16, 20,31,32, 44, 60, 71, 72, 80,81,99,253,255,256,259, 52]
    living = [30,254]
    vehicle = [10,13,18,252,257,258]
    objects = [51]
    ground = [40,48,49]
    buildings = [50]
    vegetation = [70]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 14
    elif label in living:
        return 2
    elif label in objects:
        return 9
    elif label in ground:
        return 24
    elif label in buildings:
        return 28
    elif label in vegetation:
        return 30
    else:
        if label >= 0:
            print(f'WARNING CLASS LABEL {label} not handled!!!!')
        return 0    
    

In [49]:
n_clouds = len(all_sem_clouds)
for feature in tqdm(range(n_clouds)):
    n_rows = all_sem_clouds[feature].shape[0]
    for i in range(n_rows):        
        cur_class = int(all_sem_clouds[feature][i, idx_sem])
        mapped_class = map_kitti_to_nuscenes_label(cur_class)
        all_sem_clouds[feature][i, idx_sem] = mapped_class

  0%|          | 0/500 [00:00<?, ?it/s]

In [51]:
for i in range(0, n_scans):
    filename = f"{export_ds}/91/scan{i}.npy"
    np.save(filename, all_sem_clouds[i])